# Boston Housing Classification Random Forest

In [ ]:
import sys
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
sys.path.append("..")

In [ ]:
inputFile = "../data/Boston_Housing_Data.csv"

## DataFrame creation using an ifered Schema 

In [ ]:
df = pd.read_csv(inputFile,delimiter=";")

print(df.info())

## Feature selection

In [ ]:
df_features = df.drop(["MEDV","CAT"],axis=1) # drop label attribute from the features
df_labels = df[["CAT"]].copy()
display(df_features)
display(df_labels)

## Data preparation

In [ ]:
num_column_names = df_features.select_dtypes(include=np.number).columns.tolist()
df_stand = df_features.copy()

print (num_column_names)
scaler = StandardScaler()
df_stand[num_column_names] = scaler.fit_transform(df_stand[num_column_names])
display(df_stand)

## Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_stand,df_labels,test_size=0.3,random_state=1234)
display (X_train)
display (X_test) 
display (y_train)
display (y_test)

## Random Forest Classifier

In [ ]:
rf = RandomForestClassifier(n_estimators=10,criterion="gini",max_features="sqrt",random_state=1234)

Train the model 

In [ ]:
rfModel = rf.fit(X_train,y_train["CAT"])

Test the model

In [ ]:
y_pred = rfModel.predict(X_test)
print (y_pred)

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
print("Test Error = " ,(1.0 - accuracy))